In [2]:
import requests
import json
import openai

openai.api_key = open(".tokens/openai", "r").read()
print("Started")

Started


In [3]:
# Uses anki-connect to connect to anki
response = requests.post('http://localhost:8765', data='{"action": "deckNames", "version": 6}', headers={'Content-type': 'application/json'})
print(response.json())

{'result': ['Driving License', 'Driving License Other Side', 'English', 'German', 'German::A2_Wortliste_Goethe', 'German::B1_Wortliste_DTZ_Goethe', 'German::Deutsch B1', 'German::Deutsch B1+', 'German::DW Nicos Weg B1', 'Medizin', 'Standard'], 'error': None}


In [4]:
def invoke(method, **params):
    request_data = json.dumps({
        'action': method,
        'params': params,
        'version': 6
    })

    response = requests.post('http://localhost:8765', data=request_data)
    response_data = json.loads(response.text)
    print(response_data)

    if response_data.get('error'):
        raise Exception(f"Error: {response_data['error']}")
    else:
        return response_data['result']

def clean_content(markdown: str):
    # this method extracts the code from the markdown and returns only the code from the markdown string

    result = ""
    in_code = False
    for line in markdown.split("\n"):
        if line.startswith("```"):
            in_code = not in_code
        elif in_code:
            result += line + "\n"
    return result

In [5]:
task_template = """
You create cards for Spaced Repetition to for a learner of german as a foreign language (level B1), these cards will be used in the context of preparation for the doctor appointment, please use this context, whenever it could help.

There is an existing structure for cards, for example:

```json
[    
    {
        "Wort_DE": "Termin",
        "Wort_EN": "appointment",
        "Bedeutung": "eine verabredete Zeit für eine bestimmte Tätigkeit oder ein Ereignis",
        "Artikel": "der",
        "Plural": "-e",
        "Satz1_DE": "Ich würde gern bei Ihnen einen Termin vereinbaren.",
        "Satz1_EN": "I would like to make an appointment with you.",
        "Satz2_DE": "Haben Sie für nächste Woche noch einen freien Termin?",
        "Satz2_EN": "Do you have a free appointment next week?",
        "Hinweis": "Es ist oft nötig, im Voraus einen Termin beim Arzt zu vereinbaren, besonders wenn es sich nicht um einen Notfall handelt."
    },
    {
        "Wort_DE": "um ... gehen",
        "Wort_EN": "be about, concern",
        "Bedeutung": "sich mit einer bestimmten Sache befassen oder auf sie beziehen",
        "Satz1_DE": "Worum geht's?",
        "Satz1_EN": "What's it about?",
        "Satz2_DE": "Worum geht es in diesem Film?",
        "Satz2_EN": "What is this movie about?",
        "Hinweis": "Diese Frage wird oft genutzt, um den Grund für etwas zu erfahren, zum Beispiel bei einem Arztbesuch das Anliegen des Patienten."
    }
]
```

Please use B1 Language level in the Bedeutung section.

Please add any information relevant for the doctor appointment and relevant information in the "hinweis" field, whatever is connected with a given word, and could help in preparation for the exam.

Please convert words to the infinitive form (verbs to infinitive, nouns to the singular and whatever else is applicable). But please, don't change the sentences and phrases that consist of multiple words.

Use the same JSON structure for the new words and phrases:
"""

In [17]:
cards_to_create_string = """

"""

# Split by lines and remove empty lines
cards_to_create = [line for line in cards_to_create_string.split("\n") if line != ""]

In [19]:
# We are splitting list on 4 elements chunks
# and then we are creating a list of lists
cards_to_create_chunks = [cards_to_create[x:x+4] for x in range(0, len(cards_to_create), 4)]

for chunk in cards_to_create_chunks:
    task = task_template + "\n".join(chunk)

    result = openai.ChatCompletion.create(
    model="gpt-4-1106-preview",
    messages=[
            {"role": "user", "content": task}
        ]
    )
    content = result.to_dict_recursive()['choices'][0]["message"]["content"]

    # print(content)
    cleaned_result = clean_content(content)
    # print(cleaned_result)
    new_words = json.loads(cleaned_result)
    print(new_words)

    # Remove None fields
    for word in new_words:
        for key in list(word.keys()):
            if word[key] is None:
                del word[key]
        invoke('addNote', 
        note={
            'deckName': 'Medizin', 
            'modelName': 'Deutsch, Englisch, Bedeutung', 
            'fields': word, 
            'options': {'allowDuplicate': True}, 
            'tags': ['German', 'B1', 'Medizin']})


[{'Wort_DE': 'andauernder Schmerz', 'Wort_EN': 'persistent pain', 'Bedeutung': 'Schmerz, der ununterbrochen über eine längere Zeit hinweg besteht', 'Artikel': 'der', 'Plural': '-en', 'Satz1_DE': 'Ich habe einen andauernden Schmerz in meinem Rücken.', 'Satz1_EN': 'I have a persistent pain in my back.', 'Satz2_DE': 'Der Arzt fragt, ob der Schmerz andauernd ist oder kommt und geht.', 'Satz2_EN': 'The doctor asks if the pain is persistent or comes and goes.', 'Hinweis': 'Bei der Beschreibung von Symptomen ist es wichtig, die Dauer und Konstanz des Schmerzes zu erwähnen.'}, {'Wort_DE': 'stechender Schmerz', 'Wort_EN': 'stabbing pain', 'Bedeutung': 'scharfer, intensiver Schmerz, der plötzlich und intensiv empfunden wird', 'Artikel': 'der', 'Plural': '-en', 'Satz1_DE': 'Ich verspüre einen stechenden Schmerz im Brustbereich.', 'Satz1_EN': 'I feel a stabbing pain in the chest area.', 'Satz2_DE': 'Bei bestimmten Bewegungen tritt ein stechender Schmerz in meinem Knie auf.', 'Satz2_EN': 'A stabbin

In [21]:
print(cleaned_result)

[
    {
        "Wort_DE": "Termin",
        "Wort_EN": "appointment",
        "Bedeutung": "eine verabredete Zeit für eine bestimmte Tätigkeit oder ein Ereignis",
        "Artikel": "der",
        "Plural": "-e",
        "Satz1_DE": "Ich würde gern bei Ihnen einen Termin vereinbaren.",
        "Satz1_EN": "I would like to make an appointment with you.",
        "Satz2_DE": "Haben Sie für nächste Woche noch einen freien Termin?",
        "Satz2_EN": "Do you have a free appointment next week?",
        "Hinweis": "Es ist oft nötig, im Voraus einen Termin beim Arzt zu vereinbaren, besonders wenn es sich nicht um einen Notfall handelt."
    },
    {
        "Wort_DE": "um ... gehen",
        "Wort_EN": "be about, concern",
        "Bedeutung": "sich mit einer bestimmten Sache befassen oder auf sie beziehen",
        "Satz1_DE": "Worum geht's?",
        "Satz1_EN": "What's it about?",
        "Satz2_DE": "Worum geht es in diesem Film?",
        "Satz2_EN": "What is this movie about?",
    

In [5]:
result = openai.ChatCompletion.create(
  model="gpt-4-1106-preview",
  messages=[
        {"role": "user", "content": task}
    ]
)
content = result.to_dict_recursive()['choices'][0]["message"]["content"]

In [9]:
# print(content)
cleaned_result = clean_content(content)
# print(cleaned_result)
new_words = json.loads(cleaned_result)
print(new_words)

[{'Wort_DE': 'Wartung', 'Wort_EN': 'Maintenance', 'Artikel': 'die', 'Plural': '-en', 'Satz1_DE': 'Regelmäßige Wartung des Fahrzeugs ist für die Verkehrssicherheit essenziell.', 'Satz1_EN': 'Regular maintenance of the vehicle is essential for road safety.', 'Satz2_DE': 'Während der Wartung werden verschiedene Flüssigkeitsstände und die Bremsen geprüft.', 'Satz2_EN': 'During maintenance, various fluid levels and the brakes are checked.', 'hinweis': 'Eine ordnungsgemäße Wartung des Autos ist notwendig, um sicher am Straßenverkehr teilzunehmen und ist Teil der Prüfungsfragen.'}, {'Wort_DE': 'Radbefestigung', 'Wort_EN': 'Wheel fixation, wheel mounting', 'Artikel': 'die', 'Plural': 'Radbefestigungen', 'Satz1_DE': 'Die Radbefestigung der Vorderräder muss regelmäßig überprüft werden, um Unfälle zu vermeiden.', 'Satz1_EN': 'The wheel fixation of the front wheels must be regularly checked to prevent accidents.', 'Satz2_DE': 'Eine lockere Radbefestigung kann zu schweren Lenkproblemen führen.', 'S

In [11]:
# Remove None fields
for word in new_words:
    for key in list(word.keys()):
        if word[key] is None:
            del word[key]
    invoke('addNote', 
       note={
           'deckName': 'Medizin', 
           'modelName': 'Deutsch, Englisch, Bedeutung', 
           'fields': word, 
           'options': {'allowDuplicate': True}, 
           'tags': ['German', 'B1']})


{'result': 1704640185994, 'error': None}
{'result': 1704640186022, 'error': None}
{'result': 1704640186043, 'error': None}
{'result': 1704640186067, 'error': None}
